In [0]:
from influxdb import InfluxDBClient
client = InfluxDBClient(
        host=influx_host,
        port=influx_port,
        username=influx_username,
        password=influx_password
    )
client.switch_database(influx_database)

# List all measurements in the selected database
measurements = client.get_list_measurements()

# Print the list of measurements
if measurements:
    for measurement in measurements:
        print(measurement)

In [0]:
import json
import requests
from influxdb import InfluxDBClient

# Configuration variables
url = 'https://dsi-mcs-prod.sero.wh.rnd.internal.ericsson.com/rest/volume/list?columns=n,mountdir,logicalUsed,quota'
auth = ('username', 'temppwd')  #MapR MCS Credentials
influx_host = 'localhost'
influx_port = 8086
influx_username = ''
influx_password = ''
influx_database = ''

def fetch_volume_data(url, auth):
    """Fetch volume data from the API."""
    response = requests.get(url, auth=auth)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

def prepare_influxdb_data(volumes):
    """Prepare data for InfluxDB insertion."""
    influx_data = []
    for volume in volumes:
        point = {
            "measurement": "mapr_volumes",
            "tags": {
                "volumeName": volume['volumename'],
                "mountdir": volume['mountdir']
            },
            "fields": {
                "currentUsage": int(volume['logicalUsed']),
                "currentQuota": int(volume['quota'])
            }
        }
        influx_data.append(point)
    return influx_data

def delete_all_volume_data(client):
    """Delete all data from the mapr_volumes measurement."""
    delete_query = "DELETE FROM mapr_volumes"
    client.query(delete_query)
    print("Existing data deleted from mapr_volumes database.")

def insert_data_to_influxdb(influx_data, influx_host, influx_port, influx_username, influx_password, influx_database):
    """Insert data into InfluxDB."""
    client = InfluxDBClient(
        host=influx_host,
        port=influx_port,
        username=influx_username,
        password=influx_password
    )
    client.create_database(influx_database)  # Create the database if it doesn't exist
    client.switch_database(influx_database)

    # Delete all existing records in mapr_volumes
    delete_all_volume_data(client)

    # Insert the latest data
    client.write_points(influx_data, time_precision='s', batch_size=1000, protocol='json')

    print("Latest Data successfully written to InfluxDB.")
    client.close()

def main():
    # Step 1: Fetch volume data from the API
    data = fetch_volume_data(url, auth)
    
    if data:
        # Step 2: Extract the 'data' section
        volumes = data['data']
        
        # Step 3: Prepare data for InfluxDB
        influx_data = prepare_influxdb_data(volumes)
        
        # Step 4: Insert data into InfluxDB
        insert_data_to_influxdb(influx_data, influx_host, influx_port, influx_username, influx_password, influx_database)

if __name__ == "__main__":
    main()

